Sources Used:
- http://www.bespokebots.com/betfair-ssl-certs.php
- https://betfair-datascientists.github.io/api/apiPythontutorial/

In [13]:
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json
import git

In [14]:
# loading in api login credentials
repo = git.Repo('.', search_parent_directories=True)
logins_dir = str(repo.working_tree_dir) + "/api_logins.txt"

with open(logins_dir) as f:
    login_dict =  json.load(f)

In [15]:
trading = betfairlightweight.APIClient(username=login_dict['my_username'],
                                       password=login_dict['my_password'],
                                       app_key=login_dict['my_app_key'],
                                       certs=login_dict['certs_path'])

In [16]:
trading.login()

<LoginResource>

In [227]:
# # make event type request to find horse racing event type

# horse_racing_event_type_id = trading.betting.list_event_types(
#     filter=filters.market_filter(
#         text_query='Horse Racing'
#     )
# )

# for event_type in horse_racing_event_type_id:
#     print(event_type.event_type.id, event_type.event_type.name, event_type.market_count) # prints id, name and market count
#     horse_racing_id = event_type.event_type.id

#     # list all horse racing market catalogues
#     market_catalogues = trading.betting.list_market_catalogue(
#         filter=filters.market_filter(
#             event_type_ids=[horse_racing_id],  # filter on just horse racing
#             market_countries=['GB'],  # filter on just GB countries
#             market_type_codes=['WIN'],  # filter on just WIN market types
#         ),
        
#         market_projection=['MARKET_START_TIME', 'RUNNER_DESCRIPTION'],  # runner description required
#         max_results=100
#     )

#     print('%s market catalogues returned' % len(market_catalogues))

7 Horse Racing 692
37 market catalogues returned


In [228]:
def process_runner_books(runner_books):
    best_back_prices = [runner_book.ex.available_to_back[0].price
                        if runner_book.ex.available_to_back
                        else 1.01
                        for runner_book
                        in runner_books]
    best_back_sizes = [runner_book.ex.available_to_back[0].size
                       if runner_book.ex.available_to_back
                       else 1.01
                       for runner_book
                       in runner_books]

    best_lay_prices = [runner_book.ex.available_to_lay[0].price
                       if runner_book.ex.available_to_lay
                       else 1000.0
                       for runner_book
                       in runner_books]
    best_lay_sizes = [runner_book.ex.available_to_lay[0].size
                      if runner_book.ex.available_to_lay
                      else 1.01
                      for runner_book
                      in runner_books]

    selection_ids = [runner_book.selection_id for runner_book in runner_books]
    last_prices_traded = [runner_book.last_price_traded for runner_book in runner_books]
    total_matched = [runner_book.total_matched for runner_book in runner_books]
    statuses = [runner_book.status for runner_book in runner_books]
    scratching_datetimes = [runner_book.removal_date for runner_book in runner_books]
    adjustment_factors = [runner_book.adjustment_factor for runner_book in runner_books]

    df = pd.DataFrame({
        'Selection ID': selection_ids,
        'Best Back Price': best_back_prices,
        'Best Back Size': best_back_sizes,
        'Best Lay Price': best_lay_prices,
        'Best Lay Size': best_lay_sizes,
        'Last Price Traded': last_prices_traded,
        'Total Matched': total_matched,
        'Status': statuses,
        'Removal Date': scratching_datetimes,
        'Adjustment Factor': adjustment_factors
    })
    return df

In [240]:
# market_IDs = []
# market_start_times = []
# market_names = []
# runner_dict = {}

# for market_catalogue in market_catalogues:
#     market_IDs = market_IDs + [market_catalogue.market_id]
#     market_names = market_names + [market_catalogue.market_name]
#     market_start_times.append(market_catalogue.market_start_time)
    
#     for runner in market_catalogue.runners:
#         runner_dict[runner.selection_id] = runner.runner_name
    
# GB_runners_df = pd.DataFrame()  # create an empty dataframe to append other dfs to

# # market book request
# for market,time, name in zip(market_IDs, market_start_times, market_names):
    
#     # Create a price filter. Get all traded and offer data
#     price_filter = filters.price_projection(
#         price_data=['EX_BEST_OFFERS']
#     )
    
#     market_books = trading.betting.list_market_book(
#         market_ids=[market],
#         price_projection=price_filter
#     )
    
#     # append the new market book runners to the final dataframe
#     market_book = market_books[0]
#     runners_df = process_runner_books(market_book.runners)
#     runners_df['Event_ID'] = market
#     runners_df['Event Time'] = time
#     runners_df['Event Name'] = name
#     GB_runners_df = GB_runners_df.append(runners_df, ignore_index=True)

In [241]:
market_IDs;
market_start_times;
runner_dict;
market_names;

In [254]:
GB_runners_df['Selection Name'] = GB_runners_df['Selection ID'].map(runner_dict)

In [260]:
# GB_runners_df.head(30)

In [ ]:
# include irish races as well?

In [ ]:
# creating df for each event and appending to list - > producing list of dataframes for each event

In [251]:
# speed this up - there are ways quicker than for loop but see what is needed first. 
event_dfs = []

for event_id in market_IDs:
    event_dfs.append(GB_runners_df.loc[GB_runners_df['Event_ID'] == event_id])
    

In [336]:
event_dfs[3]

,Selection ID,Best Back Price,Best Back Size,Best Lay Price,Best Lay Size,Last Price Traded,Total Matched,Status,Removal Date,Adjustment Factor,Event_ID,Event Time,Event Name,Selection Name
29,23405257,2.04,484.71,2.06,167.59,2.04,0.0,ACTIVE,None,40.378,1.165683273,2019-11-30 12:05:00,2m Listed Hrd,Floressa
30,21534802,3.25,21.67,3.35,30.50,3.25,0.0,ACTIVE,None,38.184,1.165683273,2019-11-30 12:05:00,2m Listed Hrd,Silver Forever
31,20681865,8.00,10.00,8.40,22.28,8.40,0.0,ACTIVE,None,9.698,1.165683273,2019-11-30 12:05:00,2m Listed Hrd,Jeremys Flame
32,4003026,23.00,25.48,30.00,13.55,23.00,0.0,ACTIVE,None,1.826,1.165683273,2019-11-30 12:05:00,2m Listed Hrd,Early Morning Rain
33,20347214,32.00,12.98,40.00,43.90,34.00,0.0,ACTIVE,None,5.072,1.165683273,2019-11-30 12:05:00,2m Listed Hrd,Heaven Help Us
34,23899573,65.00,19.46,990.00,4.52,120.00,0.0,ACTIVE,None,1.826,1.165683273,2019-11-30 12:05:00,2m Listed Hrd,Martha Brae
35,19358163,75.00,12.48,240.00,10.08,130.00,0.0,ACTIVE,None,1.826,1.165683273,2019-11-30 12:05:00,2m Listed Hrd,Valse Au Taillons
36,18172592,80.00,10.77,300.00,10.04,100.00,0.0,ACTIVE,None,1.190,1.165683273,2019-11-30 12:05:00,2m Listed Hrd,Nikap
